In [27]:
!pip install yfinance

In [28]:
import yfinance as yf

# Download historical stock data
data = yf.download("AAPL", start="2010-01-01", end="2023-01-01", interval="1d")
data.reset_index(inplace=True)

# Save data to a CSV file
csv_file_path = 'aapl_stock_data.csv'
data.to_csv(csv_file_path, index=False)

[*********************100%***********************]  1 of 1 completed


In [29]:
import boto3

# Initialize S3 client
s3 = boto3.client('s3')

# Define S3 bucket name and object name
bucket_name = 'time-series-analysis-pb'
s3_file_path = 'data/aapl_stock_data.csv'

# Upload the file
s3.upload_file(csv_file_path, bucket_name, s3_file_path)

print(f'File uploaded to s3://{bucket_name}/{s3_file_path}')


File uploaded to s3://time-series-analysis-pb/data/aapl_stock_data.csv


In [30]:
!pip install --upgrade sagemaker

In [31]:
import sagemaker
from sagemaker.amazon import DeepAR
from sagemaker.inputs import TrainingInput

# Initialize SageMaker session and role
role = sagemaker.get_execution_role()
session = sagemaker.Session()

# Set up the DeepAR Estimator using the built-in algorithm
deepar_estimator = DeepAR(
    role=role,
    instance_count=1,
    instance_type='ml.c4.xlarge',
    output_path=f"s3://{bucket}/output",
    sagemaker_session=session
)

# Set hyperparameters for DeepAR
deepar_estimator.set_hyperparameters(
    time_freq='D',  # daily data
    context_length=30,  # number of previous days to base the forecast on
    prediction_length=7,  # number of days to forecast
    epochs=400
)

# Define input data channels
train_data = f's3://{bucket}/data/aapl_stock_data.csv'
s3_input_train = TrainingInput(s3_data=train_data, content_type="csv")

# Train the model
deepar_estimator.fit({'train': s3_input_train})


ImportError: cannot import name 'DeepAR' from 'sagemaker.amazon' (/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/amazon/__init__.py)

In [26]:
# Deploy the trained model
predictor = deepar_estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

# Make a prediction
import json
test_data = {
    "instances": [[...]]  # provide your test data here
}
response = predictor.predict(json.dumps(test_data))


ValueError: Estimator is not associated with a training job

In [ ]:
import streamlit as st

# Historical vs forecasted data visualization
st.title('Time Series Forecasting Dashboard')

st.line_chart(historical_data)
st.line_chart(forecasted_data)

In [ ]:
from sklearn.metrics import mean_absolute_error

# Calculate MAE between historical and forecasted data
mae = mean_absolute_error(historical_values, forecast_values)
print(f"Mean Absolute Error: {mae}")
